In [ ]:
import pickle

import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

## НАСТРОЙКА

In [ ]:
TARGET_NAME = #колонка с таргетом
IMAGES_DIR = #папка с картинками
DATASET_PATH = #путь к датасету полный
FILENAME_COL = #колонка с именем файла
TARGET_SIZE = #tuple, размер ресайзинга
BATCH_SIZE = #размер батча
EPOCHS = #кол-во эпох
STEPS_PER_EPOCH = #шаги за эпоху
VALIDATION_STEPS = #шаги валидации
INUT_SHAPE = #размер входа для резнет

In [ ]:
optimizer = Adam(learning_rate=learning_rate)


def load_train(path):
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       validation_split=0.25,
                                       horizontal_flip=True,
                                       rotation_range=30,)
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=pd.read_csv(DATASET_PATH),
        directory=IMAGES_DIR,
        x_col=TARGET_NAME,
        y_col=IMAGES_PATH,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='raw',
        subset='training',
        class_mode='raw',
        seed=12345)

    return train_gen_flow

def load_test(path):
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=pd.read_csv(DATASET_PATH),
        directory=IMAGES_DIR,
        x_col=TARGET_NAME,
        y_col=IMAGES_PATH,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='raw',
        subset='validation',
        class_mode='raw',
        seed=12345)

    return test_gen_flow

def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape,
                        weights='imagenet',
                        include_top=False)
    
    backbone.trainable = False

    model = Sequential()
    model.add(backbone)
    #новая голова
    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1, activation='softmax'))
    model.compile(loss=BinaryCrossentropy(), optimizer=optimizer,
                  metrics=['accuracy'])

    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=10,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, epochs=epochs, shuffle=True)

    return model


In [ ]:
model = train_model(
    create_model(INUT_SHAPE),
    load_train(),
    load_test(),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS
    )

In [ ]:
pickle.dump(model, open('baseline.pkl', 'wb'))